In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [3]:
df = pd.read_csv('C:/Users/Kovács Márton Attila/Documents/bepps.csv', sep=';')

C:\Users\Kovács Márton Attila\AppData\Local\Temp\ipykernel_32152\1129319405.py:1: DtypeWarning: Columns (4,25,39,156,209,326,339,414,417,436,440,441) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:/Users/Kovács Márton Attila/Documents/bepps.csv', sep=';')


az adatok 2018-ból származnak, covid miatt. 

használt változók:

	a14y-év
	a1-country
	d3c-direkt export
	b2b-Owned By Private Foreign Individuals, Companies or organizations
	b8- okmány
	b7-top meneger exp in the field
	b5- Year Establishment Began Operations 
	d30a, d30b - transport and tariffs
	a4a, a6a-sector and size


dummy amik nincsenek benne a regresszióban:
	country: country_112
	transport:0
	sector:1
	size:1
	tariff:0


In [4]:
ex = df
ex = ex[['a14y', 'a1', 'd3c', 'b2b', 'b8', 'b7', 'b5', 'd30a', 'a4a', 'a6a', 'd30b', 'b6']]
ex = ex[ex['a14y'] == 2018 | 2019]
ex.dropna(inplace = True)
ex = ex[ex['b6'] > 10]
ex.drop('b6', axis = 1, inplace = True)
ex = ex[ex['d3c'] > 0]
ex = ex[ex['b2b'] >= 0]
ex = ex[ex['b8'] > 0]
ex = ex[(ex['b7'] >= 0) & (ex['b7'] < 60)]
ex = ex[ex['b5'] > 0]
ex = ex[ex['d30a'] >= 0]
ex = ex[ex['d30a'] >= 0]
ex = ex[ex['d30b'] >= 0]
ex = ex[ex['a4a'] >= 0]
ex = ex[(ex['a6a'] > 0) & (ex['a6a'] < 4)]
ex = ex.drop('a14y', axis='columns')
ex = ex.rename({'a1': 'country', 
               'd3c': 'dexp',
               'b2b': 'foreign_owned',
               'b8': 'cert',
               'b7': 'meneger_exp',
               'b5': 'first_year',
               'd30a': 'transport',
               'd30b': 'tariff',
               'a4a':'sector',
               'a6a':'size'}, axis = 'columns')

data = ex

ex['foreign_owned_dummy'] = ex['foreign_owned'].apply(lambda x: 1 if x > 50 else 0)
ex_dummies = pd.get_dummies(ex[['country', 'transport', 'sector', 'size', 'tariff']].astype(str))
ex = ex.merge(ex_dummies, how='outer', left_index = True, right_index = True)
ex = ex.drop(['country', 'country_112','foreign_owned','transport','transport_0.0','sector','sector_1.0','size','size_1.0','tariff','tariff_0.0'], axis = 'columns')
ex['age'] = 2023 - ex['first_year']
ex.drop('first_year',axis=1,inplace=True)
ex = ex.astype('int64')

ex['intercept'] = 1
ex_x = ex.drop('dexp', axis = 1)
ex_y = np.array(ex['dexp'])
olsresults = sm.OLS(ex_y, ex_x).fit()
print(olsresults.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.277
Model:                            OLS   Adj. R-squared:                  0.251
Method:                 Least Squares   F-statistic:                     10.74
Date:                Wed, 17 May 2023   Prob (F-statistic):          3.25e-102
Time:                        11:03:46   Log-Likelihood:                -10552.
No. Observations:                2207   AIC:                         2.126e+04
Df Residuals:                    2130   BIC:                         2.170e+04
Df Model:                          76                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
cert                    1.1011    

In [5]:
print(data.describe())

           country         dexp  foreign_owned         cert  meneger_exp  \
count  2207.000000  2207.000000    2207.000000  2207.000000  2207.000000   
mean     73.894427    48.277300      19.289986     1.421840    22.249660   
std      29.590989    33.935117      36.730441     0.493965    11.624478   
min      37.000000     1.000000       0.000000     1.000000     1.000000   
25%      54.000000    20.000000       0.000000     1.000000    15.000000   
50%      66.000000    40.000000       0.000000     1.000000    20.000000   
75%      78.000000    80.000000       2.000000     2.000000    30.000000   
max     160.000000   100.000000     100.000000     2.000000    59.000000   

        first_year    transport       sector         size       tariff  \
count  2207.000000  2207.000000  2207.000000  2207.000000  2207.000000   
mean   1995.061169     1.193022     6.186679     2.446307     1.132759   
std      18.643732     1.257754     4.764970     0.673749     1.201786   
min    1817.000000 